In [6]:
from h2ogpte import H2OGPTE

import pandas as pd

import json

In [7]:
!ls tmp/data

movie_rating_modelingdata.csv
movie_rating_modelingdata_predictions.csv
movies_sample.csv
movies_sample_AllAttributes.csv
ratings_sample.csv
user_sample.csv
user_sample_AllAttributes.csv


In [10]:
df = pd.read_csv('tmp/data/movies_sample.csv')
df

,movieId,title,genres
0,1,Toy Story (1995),"['Adventure', 'Animation', 'Children', 'Comedy..."
1,47,Seven (a.k.a. Se7en) (1995),"['Mystery', 'Thriller']"
2,50,"Usual Suspects, The (1995)","['Crime', 'Mystery', 'Thriller']"
3,110,Braveheart (1995),"['Action', 'Drama', 'War']"
4,150,Apollo 13 (1995),"['Adventure', 'Drama', 'IMAX']"
5,260,Star Wars: Episode IV - A New Hope (1977),"['Action', 'Adventure', 'Sci-Fi']"
6,296,Pulp Fiction (1994),"['Comedy', 'Crime', 'Drama', 'Thriller']"
7,318,"Shawshank Redemption, The (1994)","['Crime', 'Drama']"
8,356,Forrest Gump (1994),"['Comedy', 'Drama', 'Romance', 'War']"
9,480,Jurassic Park (1993),"['Action', 'Adventure', 'Sci-Fi', 'Thriller']"


In [3]:
with open('../_secret') as f:
    key = json.load(f)

In [4]:
client = H2OGPTE(
    address='https://playground.h2ogpte.h2o.ai',
    api_key=key['client-access-test'],
)

client

In [5]:
client.get_llm_names()

['mistralai/Mixtral-8x7B-Instruct-v0.1',
 'meta-llama/Meta-Llama-3-8B-Instruct',
 'meta-llama/Meta-Llama-3-70B-Instruct',
 'nvidia/Llama3-ChatQA-1.5-70B',
 'NousResearch/Nous-Capybara-34B',
 'mistralai/Mistral-7B-Instruct-v0.3',
 'h2oai/h2o-danube2-1.8b-chat',
 'mistralai/Mistral-Nemo-Instruct-2407',
 'OpenGVLab/InternVL-Chat-V1-5',
 'THUDM/cogvlm2-llama3-chat-19B',
 'liuhaotian/llava-v1.6-34b',
 'lmms-lab/llama3-llava-next-8b',
 'mistral-small-latest',
 'mistral-large-latest',
 'mistral-medium',
 'claude-3-5-sonnet-20240620',
 'claude-3-sonnet-20240229',
 'claude-3-opus-20240229',
 'claude-3-haiku-20240307',
 'microsoft/Phi-3-vision-128k-instruct',
 'microsoft/Phi-3-medium-128k-instruct',
 'google/gemma-2-27b-it',
 'gemini-1.5-pro-latest',
 'gemini-1.5-flash-latest',
 'gpt-3.5-turbo-0613',
 'gpt-3.5-turbo-16k-0613',
 'gpt-35-turbo-1106',
 'gpt-4-1106-preview',
 'gpt-4-turbo-2024-04-09']

In [15]:
chat_session_id = client.create_chat_session_on_default_collection()
chat_session_id

'0dfdb46d-3901-42ef-a297-e3de87143d8b'

In [24]:
SYS_PROMPT = '''
あなたはtitleとgenresから、映画紹介文を作成する映画評論AIです。文章は必ず日本語で作成します。
'''

title = df['title'][0]
genres = df['genres'][0]

MESSAGE = '''
genresを参考にしてtitleの映画の紹介文を400から500字程度で考えて下さい。
title= {}
genres= {}

注意点として、紹介文の中にはTitleを含まないでください。以下の例参照。

例: 
OK: この映画は、...
NG: titleは、...

'''.format(title, genres)

print(MESSAGE)
print('---------------------')

MODEL = 'mistralai/Mixtral-8x7B-Instruct-v0.1'

with client.connect(chat_session_id) as session:
    reply = session.query(
        message = MESSAGE,
        system_prompt=SYS_PROMPT,
        llm=MODEL,
        rag_config={"rag_type": "llm_only",},
    )
    print(reply.content)


genresを参考にしてtitleの映画の紹介文を400から500字程度で考えて下さい。
title= Toy Story (1995)
genres= ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy']

注意点として、紹介文の中にはTitleを含まないでください。以下の例参照。

例: 
OK: この映画は、...
NG: titleは、...


---------------------
この映画は、1995年に公開されたアドベンチャー・アニメーション映画です。子ども向けのコメディーとファンタジー要素が融合し、皆さまにとって楽しい時間を提供してくれます。

主人公は、木製の玩具だ。彼らは、人間の子どもたちが見逃すほかの世界で生きています。彼らは、子どもたちの遊びに堪能するほか、彼ら自身の世界を作り出しています。

この映画では、木製の玩具たちが、人間の子どもたちから見逃されることを恐れ、彼ら自身の世界を守るために協力し合います。彼らは、人間の子どもたちが見逃すほかの世界で、彼ら自身の物語を作り出していきます。

この映画は、簡単なストーリーと、面白いジオーグとバズ・ライトの二人の関係が、子どもたちにとって楽しい時間を提供してくれます。また、大人たちにも、子どもたちの心の中で起きていることについて、深い気づきを与えてくれます。

この映画は、アニメーションの技術も魅力的で、木製の玩具たちが生き生きするように見えます。彼らの動きや表情は、子どもたちにとって、とても面白く、大人たちにとっても、感動的です。

この映画は、子どもたちにとって、楽しい時間を提供してくれます。また、大人たちにも、子どもたちの心の中で起きていることについて、深い気づきを与えてくれます。この映画は、アドベンチャー、アニメーション、子ども向けのコメディー、ファンタジーのすべてを含み、皆さまにとって楽しい時間を提供してくれます。


In [25]:
SYS_PROMPT = '''
あなたはtitleとgenresから、映画紹介文を作成する映画評論AIです。文章は必ず日本語で作成します。
'''

title = df['title'][0]
genres = df['genres'][0]

MESSAGE = '''
genresを参考にしてtitleの映画の紹介文を400から500字程度で考えて下さい。
title= {}
genres= {}

注意点として、紹介文の中にはTitleを含まないでください。以下の例参照。

例: 
OK: この映画は、...
NG: titleは、...

'''.format(title, genres)

print(MESSAGE)
print('---------------------')

MODEL = 'gpt-3.5-turbo-0613'

with client.connect(chat_session_id) as session:
    reply = session.query(
        message = MESSAGE,
        system_prompt=SYS_PROMPT,
        llm=MODEL,
        rag_config={"rag_type": "llm_only",},
    )
    print(reply.content)


genresを参考にしてtitleの映画の紹介文を400から500字程度で考えて下さい。
title= Toy Story (1995)
genres= ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy']

注意点として、紹介文の中にはTitleを含まないでください。以下の例参照。

例: 
OK: この映画は、...
NG: titleは、...


---------------------
この映画は、1995年に公開されたアドベンチャー、アニメーション、子供向け、コメディ、ファンタジーの要素を持つ作品です。主人公は、おもちゃの世界で冒険を繰り広げるウッディというカウボーイの人形と、彼の仲間であるバズ・ライトイヤーという宇宙人のおもちゃです。彼らは、主人公の少年アンディのおもちゃとして暮らしています。

物語は、アンディが新しいおもちゃとしてバズを手に入れたことから始まります。ウッディはバズに嫉妬し、自分がアンディのお気に入りのおもちゃであることを証明しようとします。しかし、ウッディとバズは意外な事件に巻き込まれてしまいます。彼らはアンディの家を抜け出し、おもちゃたちの世界を冒険することになります。

この映画は、子供たちだけでなく、大人も楽しめる作品です。おもちゃたちの個性豊かなキャラクターたちや、彼らが織り成す感動的なストーリーが魅力です。また、映像の美しさやユーモアのセンスも特筆されます。

「トイ・ストーリー」は、おもちゃたちの友情や成長、家族の絆を描いた感動的な作品です。子供たちに夢と冒険を与えるだけでなく、大人たちにも心温まるメッセージを届けてくれる映画です。ぜひ、家族や友人と一緒に楽しんでみてください。


In [28]:
for i, row in df.iterrows():
    print(row['title'])

Toy Story (1995)
Seven (a.k.a. Se7en) (1995)
Usual Suspects, The (1995)
Braveheart (1995)
Apollo 13 (1995)
Star Wars: Episode IV - A New Hope (1977)
Pulp Fiction (1994)
Shawshank Redemption, The (1994)
Forrest Gump (1994)
Jurassic Park (1993)
Schindler's List (1993)
Terminator 2: Judgment Day (1991)
Silence of the Lambs, The (1991)
Independence Day (a.k.a. ID4) (1996)
Star Wars: Episode V - The Empire Strikes Back (1980)
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
Matrix, The (1999)
American Beauty (1999)
Fight Club (1999)
Lord of the Rings: The Fellowship of the Ring, The (2001)


In [35]:
MODEL = 'gpt-3.5-turbo-0613'

SYS_PROMPT = '''
あなたはtitleとgenresから、映画紹介文を作成する映画評論AIです。文章は必ず日本語で作成します。
'''

sentences = []

for i, row in df.iterrows():
    title = row['title']
    genres = row['genres']

    MESSAGE = '''
    genresを参考にしてtitleの映画の紹介文を400から500字程度で創作して下さい。
    title= {}
    genres= {}

    '''.format(title, genres)

    print(title, genres)
    print('---------------------')


    with client.connect(chat_session_id) as session:
        reply = session.query(
            message = MESSAGE,
            system_prompt=SYS_PROMPT,
            llm=MODEL,
            rag_config={"rag_type": "llm_only",},
        )
        sentences.append(reply.content)

len(sentences)

Toy Story (1995) ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy']
---------------------
Seven (a.k.a. Se7en) (1995) ['Mystery', 'Thriller']
---------------------
Usual Suspects, The (1995) ['Crime', 'Mystery', 'Thriller']
---------------------
Braveheart (1995) ['Action', 'Drama', 'War']
---------------------
Apollo 13 (1995) ['Adventure', 'Drama', 'IMAX']
---------------------
Star Wars: Episode IV - A New Hope (1977) ['Action', 'Adventure', 'Sci-Fi']
---------------------
Pulp Fiction (1994) ['Comedy', 'Crime', 'Drama', 'Thriller']
---------------------
Shawshank Redemption, The (1994) ['Crime', 'Drama']
---------------------
Forrest Gump (1994) ['Comedy', 'Drama', 'Romance', 'War']
---------------------
Jurassic Park (1993) ['Action', 'Adventure', 'Sci-Fi', 'Thriller']
---------------------
Schindler's List (1993) ['Drama', 'War']
---------------------
Terminator 2: Judgment Day (1991) ['Action', 'Sci-Fi']
---------------------
Silence of the Lambs, The (1991) ['Crime', '

20

In [36]:
sentences[18:]

['『ファイト・クラブ』（1999年）は、アクション、犯罪、ドラマ、スリラーの要素を持つ映画です。この作品は、現代社会の中で生きる男性たちの心の闇と葛藤を描いています。\n\n物語は、主人公である無名の男（エドワード・ノートン）が、自己啓発グループに参加することから始まります。彼はそこで、自分自身を見つめ直す機会を得ますが、同時に、自己啓発グループのメンバーとの関係にも悩みます。\n\nそんな中、彼はタイラー・ダーデン（ブラッド・ピット）という謎めいた男性と出会います。彼らは共に「ファイト・クラブ」という秘密の組織を作り、暴力的な格闘技を通じて自己の限界を超えようとします。しかし、次第にファイト・クラブは過激化し、社会的な混乱を引き起こすことになります。\n\nこの映画は、アクションシーンや犯罪の要素を通じてスリリングな展開を見せますが、同時に、男性たちの内面の葛藤や現代社会の問題にも深く切り込んでいます。登場人物たちの心の闇や孤独、そして絶望感がリアルに描かれており、観る者の心に強い印象を残します。\n\nまた、本作はダークで独特な雰囲気を持っており、映像美や音楽も非常に印象的です。映画の中で繰り広げられる格闘シーンは迫力満点であり、観る者を引き込んで離しません。\n\n『ファイト・クラブ』は、社会の常識や価値観に疑問を投げかける作品であり、一度観たら忘れられない映画体験を提供してくれます。アクション、犯罪、ドラマ、スリラーといったジャンルの要素を巧みに組み合わせた本作は、映画ファンならずとも必見の一本です。',
 '『ロード・オブ・ザ・リング：旅の仲間』（2001年）は、冒険とファンタジーが織りなす壮大な物語です。この映画は、J.R.R.トールキンの同名小説を基にしています。\n\n物語は、中つ国と呼ばれる神秘的な世界を舞台に展開します。主人公フロド・バギンズは、一つの指輪を持つことになります。この指輪は、全ての力を支配する力を秘めており、闇の勢力がそれを手に入れようとしています。フロドは、仲間たちと共に指輪を壊すための冒険の旅に出ることになります。\n\n彼の仲間たちは、さまざまな種族や能力を持つキャラクターで構成されています。彼らは、エルフ、ドワーフ、人間、そして魔法使いなど、さまざまな存在です。彼らは団結し、困難な試練に立ち向かいながら、指輪を壊すための旅

In [37]:
df['story'] = sentences

In [38]:
df

,movieId,title,genres,story
0,1,Toy Story (1995),"['Adventure', 'Animation', 'Children', 'Comedy...",『トイ・ストーリー』（1995年）は、冒険、アニメーション、子供向け、コメディ、ファンタジー...
1,47,Seven (a.k.a. Se7en) (1995),"['Mystery', 'Thriller']",映画『セブン』は、1995年に公開されたミステリーとスリラーの要素を持つ作品です。本作は、デ...
2,50,"Usual Suspects, The (1995)","['Crime', 'Mystery', 'Thriller']",『アンツ・アンド・ワスプ』（1995年）は、犯罪、ミステリー、スリラーの要素を巧みに組み合わ...
3,110,Braveheart (1995),"['Action', 'Drama', 'War']",『ブレイブハート』（1995年）は、アクション、ドラマ、戦争というジャンルを融合させた感動作...
4,150,Apollo 13 (1995),"['Adventure', 'Drama', 'IMAX']",『アポロ13』（1995年）は、冒険、ドラマ、IMAXの要素を持つ映画です。この作品は、実際...
5,260,Star Wars: Episode IV - A New Hope (1977),"['Action', 'Adventure', 'Sci-Fi']",『スター・ウォーズ エピソード4/新たなる希望』（1977年）は、アクション、冒険、SFの要...
6,296,Pulp Fiction (1994),"['Comedy', 'Crime', 'Drama', 'Thriller']",『パルプ・フィクション』（1994年）は、コメディ、クライム、ドラマ、スリラーの要素を巧みに...
7,318,"Shawshank Redemption, The (1994)","['Crime', 'Drama']",『ショーシャンクの空に』（1994年公開）は、犯罪とドラマの要素を巧みに組み合わせた感動作で...
8,356,Forrest Gump (1994),"['Comedy', 'Drama', 'Romance', 'War']",『フォレスト・ガンプ 一期一会』（1994年）は、コメディ、ドラマ、ロマンス、戦争という多様...
9,480,Jurassic Park (1993),"['Action', 'Adventure', 'Sci-Fi', 'Thriller']",『ジュラシック・パーク』（1993年）は、アクション、アドベンチャー、サイエンスフィクション...


In [40]:
df.to_csv('tmp/data/movies_sample_story.csv', index=False)

In [82]:
# textとして出力

txt = ''
for i, row in df.iterrows():
    txt = txt + '# Movie Title\n'
    txt = txt + row['title']
    txt = txt + '\n \n'
    txt = txt + '# Movie Detail\n'
    txt = txt + row['story']
    txt = txt + '\n \n ##########################\n'

In [86]:
#txt

In [85]:
with open('tmp/data/movies_sample_story.txt', mode='w') as f:
    f.write(txt)